# Tutorial 4: Trajectory aggregation (flow maps)

<img align="right" src="https://anitagraser.github.io/movingpandas/pics/movingpandas.png">

This tutorial covers trajectory generalization and aggregation using flow maps. 

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from datetime import datetime, timedelta

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

## Ship movements (AIS data)

In [ ]:
df = read_file('data/demodata_ais.gpkg')
df['t'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S')
df = df.set_index('t')
df = df[df.SOG>0]
df.size

In [ ]:
MIN_LENGTH = 100 # meters
traj_collection = mpd.TrajectoryCollection(df, 'MMSI', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=5))
print("Extracted {} individual trips from {} continuous vessel tracks".format(len(trips), len(traj_collection)))

Generalizing the trip trajectories significantly speeds up the following aggregation step.

In [ ]:
%%time
generalized = mpd.MinDistanceGeneralizer(trips).generalize(tolerance=100)

In [ ]:
%%time
aggregator = mpd.TrajectoryCollectionAggregator(generalized, max_distance=1000, min_distance=100, min_stop_duration=timedelta(minutes=5))

In [ ]:
pts = aggregator.get_significant_points_gdf()
clusters = aggregator.get_clusters_gdf()
( pts.hvplot(geo=True, tiles='OSM', frame_width=800) * 
  clusters.hvplot(geo=True, color='red') )

In [ ]:
flows = aggregator.get_flows_gdf()

In [ ]:
( #trips.hvplot(color='gray')  *
  flows.hvplot(geo=True, hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', tiles='OSM') * 
  clusters.hvplot(geo=True, color='red', size='n') )

### Comparison of generalized vs. original trajectories

In [ ]:
%%time
aggregator_original = mpd.TrajectoryCollectionAggregator(trips, max_distance=1000, min_distance=100, min_stop_duration=timedelta(minutes=5))

In [ ]:
( aggregator_original.get_flows_gdf().hvplot(title='Original', geo=True, tiles='OSM', hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', frame_height=400, frame_width=400) * 
  aggregator_original.get_clusters_gdf().hvplot(geo=True, color='red', size='n') +
  flows.hvplot(title='Generalized', geo=True, tiles='OSM', hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', frame_height=400, frame_width=400) * 
  clusters.hvplot(geo=True, color='red', size='n') 
)

## Bird migration data

In [ ]:
df = read_file('data/demodata_gulls.gpkg')
df['t'] = pd.to_datetime(df['timestamp'])
df = df.set_index('t')
df.size

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'individual-local-identifier', min_length=MIN_LENGTH)     
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = mpd.TemporalSplitter(traj_collection).split(mode='month')
print("Extracted {} individual trips from {} continuous tracks".format(len(trips), len(traj_collection)))

In [ ]:
generalized = mpd.MinTimeDeltaGeneralizer(trips).generalize(tolerance=timedelta(days=1))

In [ ]:
%%time
aggregator = mpd.TrajectoryCollectionAggregator(generalized, max_distance=1000000, min_distance=100000, min_stop_duration=timedelta(minutes=5))

In [ ]:
flows = aggregator.get_flows_gdf()
clusters = aggregator.get_clusters_gdf()

( flows.hvplot(geo=True, hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', tiles='OSM') * 
  clusters.hvplot(geo=True, color='red', size='n') )

In [ ]:
%%time
aggregator_original = mpd.TrajectoryCollectionAggregator(trips, max_distance=1000000, min_distance=100000, min_stop_duration=timedelta(minutes=5))

In [ ]:
( aggregator_original.get_flows_gdf().hvplot(title='Original', geo=True, tiles='OSM', hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', frame_height=600, frame_width=400) * 
  aggregator_original.get_clusters_gdf().hvplot(geo=True, color='red', size='n') +
  flows.hvplot(title='Generalized', geo=True, tiles='OSM', hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', frame_height=600, frame_width=400) * 
  clusters.hvplot(geo=True, color='red', size='n') 
)